In [1]:
import blackjack_engine
import random
import numpy as np
import pandas as pd
# random.seed(4) # can use this for repeatable results

In [2]:
def padarray(A, size):
    if len(A) == 0:
        A = ['']
    t = size - len(A)
    arr = np.pad(A, pad_width=(0, t), mode='empty')
    
    return arr.astype('<U32')

def player_bot(hand):
    """ makes random choice, or makes decisions using neural net etc"""
    # decision making code goes here
    return random.choice(['hit', 'stay']) 

def play_blackjack_game():
    # initialize a game with shuffled deck and hands dealt to dealer and player:
    game = blackjack_engine.BlackjackGame()
    moves = []
    #game_states = []
    dealer_cards = []
    player_hands = []
    game_results = []
    if game.is_finished:
        # somebody got dealt a blackjack...need to deal with that case
        pass
    while not game.is_finished: # if nobody got dealt a blackjack, loop through decisions
        player_hand = game.player_hands[0].cards[:]
        dealer_card = game.dealer_hand.cards[0]
        #game_state_row = {'player_hand':player_hand,'dealer_card':dealer_card}
        #game_state_row = [player_hand, dealer_card]
        player_hands.append(player_hand)
        dealer_cards.append(dealer_card)
        hit_or_stay = player_bot(game.player_hands)
        moves.append(hit_or_stay)
        game.player_move(hit_or_stay)

    [game_results.append(game.result) for game_result in np.arange(len(moves))]
        
    return player_hands, dealer_cards, moves, game_results

# implement a training set generation routine: give the API random actions and see how games turn out
def build_training_set(n_pts):
    '''
    state_of_play is an n x m array of values where n is the number of data points, and m is the max number of features in a hand 
    y is an n x 1 array, where n is the number of data points (number of times the player drew a card, basically) 
    '''
    all_player_hands = []
    all_dealer_cards = []
    all_moves = []
    all_game_results = []

    for pt in np.linspace(0,n_pts,n_pts+1):
        [player_hands, dealer_cards, moves,game_results] = play_blackjack_game()
        all_player_hands.append(player_hands)
        all_dealer_cards.append(dealer_cards)
        all_moves.append(moves)
        all_game_results.append(game_results)
    return all_player_hands, all_dealer_cards, all_moves, all_game_results

def flatten_list(input_list):
    flat_list = []
    for element in input_list:
        for sub_element in element:
            flat_list.append(sub_element)
    return flat_list

In [128]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import normalize

def reshape_data(input_data):
    # flatten and preprocess all of the input data
    [all_player_hands, all_dealer_cards, all_moves, all_game_results] = input_data
    flat_player_hands = flatten_list(all_player_hands)
    flat_game_results = flatten_list(all_game_results)
    flat_moves = flatten_list(all_moves)
    flat_dealer_cards = flatten_list(all_dealer_cards)
    
    # format X to be a n x m matrix where n is the number of examples and m is the max number of features
    X = []
    for instance in np.arange(len(flat_moves)):
        Xrow = [flat_dealer_cards[int(instance)], flat_game_results[int(instance)]]
        for card in flat_player_hands[int(instance)]:
            Xrow.append(card)
        while len(Xrow) < 9:
            Xrow.append('pad')
        X.append(Xrow)

    return X, flat_moves

def encode_inputs(X):
    oe = OrdinalEncoder()
    oe.fit(X)
    X_enc = oe.transform(X)

    return X_enc

def encode_outputs(y):
    le = LabelEncoder()
    le.fit(y)
    y_enc = le.transform(y)
    return y_enc

def preprocessing_pipeline(input_data):
    # separate and shape the input data
    X,y = reshape_data(input_data)
    
    # encode all variables numerically
    X = encode_inputs(X)
    y = encode_outputs(y)
    
    # normalize the inputs
    X = normalize(X)
    
    # split into test and train sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test


# Build a training set with a random player of a specified size

In [129]:
n_examples = 1000
[all_player_hands, all_dealer_cards, all_moves, all_game_results] = build_training_set(n_examples)
input_data = [all_player_hands, all_dealer_cards, all_moves, all_game_results]

# Preprocess the input data (reshape, encode, normalize)

In [130]:
X_train, X_test, y_train, y_test = preprocessing_pipeline(input_data)
print('X_train:',np.shape(X_train))
print('X_test:',np.shape(X_test))
print('y_train:',np.shape(y_train))
print('y_test:',np.shape(y_test))

X_train: (976, 9)
X_test: (244, 9)
y_train: (976,)
y_test: (244,)


In [6]:
# train the neural network on the previously generated data
def build_model(X,y):
    input_shape = np.shape(X)
    output_shape = np.shape(y) 
    # use a simple sequential dense model to predict which actions will result in a win
    return model

def train_model(untrained_model,training_data):
    training_data = [X_train, y_train, X_test, y_test]
    
    return trained_network

def use_model_to_make_decision(trained_model,state_of_play):

    decision = trained_model.predict(state_of_play)
    while game_is_still_in_play:
        state_of_play = play_blackjack_hand(state_of_play,decision)
        game_is_still_in_play = check_result(state_of_play)
    return state_of_play
    